In [ ]:
# Install condacolab
!pip install -q condacolab
import condacolab
condacolab.install()

!conda create -n internvl python=3.9 -y

!source activate internvl

In [ ]:
!git clone https://github.com/Czi24/InternVL-20240714.git
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

!pip install flash-attn==2.3.6 --no-build-isolation
!pip install transformers==4.37.2
!pip install opencv-python termcolor yacs pyyaml scipy
!pip install deepspeed==0.13.5
!pip install pycocoevalcap tqdm

%cd /content/InternVL-20240714/internvl_chat_llava/
!pip install -e .

In [26]:
!pip install huggingface_hub protobuf
from huggingface_hub import snapshot_download
%cd /content
snapshot_download(repo_id="OpenGVLab/Mini-InternVL-Chat-4B-V1-5", local_dir="Mini-InternVL-Chat-4B-V1-5")

/content


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/6.58k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

modeling_phi3.py:   0%|          | 0.00/74.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

'/content/Mini-InternVL-Chat-4B-V1-5'

In [27]:
%cd /content/InternVL-20240714/internvl_chat_llava/
import threading
import subprocess

# start Controller
threading.Thread(
    target=lambda: subprocess.run(
        ['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000'],
        check=True
    ),
    daemon=True
).start()

/content/InternVL-20240714/internvl_chat_llava


In [28]:
%cd /content/InternVL-20240714/internvl_chat
import threading
import subprocess

# Define the command to run the model worker
command = [
    'python3', '-m', 'internvl.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40001',
    '--worker', 'http://localhost:40001',
    '--model-path', '/content/Mini-InternVL-Chat-4B-V1-5'
]

# Run the command in a separate thread
thread = threading.Thread(
    target=lambda: subprocess.run(command, check=True, shell=False),
    daemon=True
)
thread.start()


/content/InternVL-20240714/internvl_chat


In [29]:
%cd /content/InternVL-20240714/internvl_chat_llava
!python3 -m llava.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share

/content/InternVL-20240714/internvl_chat_llava
2024-07-15 12:43:47 | INFO | gradio_web_server | args: Namespace(host='0.0.0.0', port=None, controller_url='http://localhost:10000', concurrency_count=10, model_list_mode='reload', share=True, moderate=False, embed=False)
2024-07-15 12:43:47 | INFO | gradio_web_server | Models: ['Mini-InternVL-Chat-4B-V1-5']
2024-07-15 12:43:47 | INFO | gradio_web_server | Namespace(host='0.0.0.0', port=None, controller_url='http://localhost:10000', concurrency_count=10, model_list_mode='reload', share=True, moderate=False, embed=False)
2024-07-15 12:43:47 | INFO | stdout | Running on local URL:  http://0.0.0.0:7860
2024-07-15 12:43:49 | INFO | stdout | Running on public URL: https://0d27314acf919717df.gradio.live
2024-07-15 12:43:49 | INFO | stdout | 
2024-07-15 12:43:49 | INFO | stdout | This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
2